# Pandas Lab — FAANG-Level Hands-On

**Goal:** Build strong intuition for Pandas transformations used in ML pipelines (joins, groupby, leakage-safe features).

**Outcome:** You can write correct, scalable Pandas code and explain common gotchas (join explosion, leakage, apply misuse).


In [ ]:
import pandas as pd
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')


## Section 0 — Synthetic Dataset (Interview-Friendly)
We’ll use synthetic tables to mirror typical ML feature engineering tasks.

In [ ]:
rng = np.random.default_rng(0)

users = pd.DataFrame({
    'user_id': [101, 102, 103, 104],
    'country': ['IN', 'US', 'IN', 'CA'],
    'signup_ts': pd.to_datetime(['2025-01-01', '2025-01-03', '2025-01-04', '2025-01-10'])
})

events = pd.DataFrame({
    'user_id': rng.choice(users['user_id'], size=30, replace=True),
    'event_ts': pd.to_datetime('2025-01-01') + pd.to_timedelta(rng.integers(0, 20, size=30), unit='D'),
    'event': rng.choice(['view', 'add_to_cart', 'purchase'], size=30, replace=True, p=[0.7, 0.2, 0.1]),
    'amount': rng.choice([0, 0, 0, 19.99, 49.99, 99.99], size=30, replace=True),
})

# Ensure amount > 0 only for purchases (simple realism)
events.loc[events['event'] != 'purchase', 'amount'] = 0

users, events.head()


## Section 1 — DataFrame Fundamentals

### Task 1.1: Basic filtering + new columns

Create:
- `purchases`: rows where `event == 'purchase'`
- add column `event_day` = date (no time)

# HINT:
- Use boolean indexing
- Use `.dt.floor('D')` or `.dt.date` (but prefer datetime64)

**Explain:** Why does vectorized `.dt` beat per-row parsing?


In [ ]:
# TODO
purchases = ...
purchases = purchases.copy()
purchases['event_day'] = ...

check('purchases_only', set(purchases['event'].unique()) <= {'purchase'})
check('event_day_dtype', str(purchases['event_day'].dtype).startswith('datetime'))


## Section 2 — GroupBy (Core for Features)

### Task 2.1: Per-user aggregates
Compute per-user features:
- `n_events`
- `n_purchases`
- `total_revenue` (sum of amount)

# HINT:
- Use `groupby('user_id').agg(...)`
- For `n_purchases`, use conditional aggregation

**FAANG gotcha:** Counting purchases using `amount>0` vs `event=='purchase'` can diverge if data is messy.


In [ ]:
# TODO
user_features = ...

user_features = user_features.reset_index()
print(user_features)
check('has_user_id', 'user_id' in user_features.columns)
check('has_total_revenue', 'total_revenue' in user_features.columns)


### Task 2.2: GroupBy transform (row-level feature)

Add a column to `events` called `user_event_count` = number of events for that user.

# HINT:
- Use `groupby(...).transform('count')`

**Explain:** Why is `transform` different from `agg`?


In [ ]:
# TODO
events2 = events.copy()
events2['user_event_count'] = ...

check('user_event_count_nonnull', events2['user_event_count'].notna().all())
check('counts_positive', (events2['user_event_count'] > 0).all())


## Section 3 — Joins & Merge (Feature Table Building)

### Task 3.1: Join user features back to users

Create `user_table` by joining `users` with `user_features` on `user_id`.

# HINT:
- Use `merge(..., how='left')`
- Fill missing aggregates with 0

**FAANG gotcha:** If you accidentally do a many-to-many join, row count explodes. Always validate row counts.


In [ ]:
# TODO
user_table = ...

# Fill NaNs for users with no events
for col in ['n_events', 'n_purchases', 'total_revenue']:
    if col in user_table.columns:
        user_table[col] = user_table[col].fillna(0)

check('rowcount_preserved', len(user_table) == len(users))
print(user_table)


### Task 3.2: Join explosion debugging (mini)

Construct a tiny example where a join becomes many-to-many and explodes rows. Then fix it.

# HINT:
- Create `left` with duplicate keys
- Create `right` with duplicate keys
- Merge and observe rowcount

**Explain:** What join cardinality do you expect (1:1, 1:n, n:1, n:n)?


In [ ]:
# TODO
left = pd.DataFrame({'k': [1, 1, 2], 'l': ['a', 'b', 'c']})
right = pd.DataFrame({'k': [1, 1, 2], 'r': ['x', 'y', 'z']})
exploded = left.merge(right, on='k', how='inner')
print('left rows', len(left), 'right rows', len(right), 'merged rows', len(exploded))

# TODO: Fix (example: deduplicate right to 1 row per k)
right_dedup = ...
fixed = left.merge(right_dedup, on='k', how='inner')
print('fixed rows', len(fixed))


## Section 4 — Time-based Features & Leakage (FAANG System Thinking)

### Task 4.1: Leakage-safe feature

For each user and each event row, compute `purchases_before` = number of purchases strictly BEFORE that `event_ts`.

Constraints:
- No Python loops over rows
- Must be time-aware

# HINT:
- Sort by user_id, event_ts
- Create boolean `is_purchase`
- Use groupby + cumsum, then shift

**FAANG gotcha:** If you include the current row in the count, you leak label info for purchase events.


In [ ]:
# TODO
events3 = events.copy().sort_values(['user_id', 'event_ts']).reset_index(drop=True)
events3['is_purchase'] = (events3['event'] == 'purchase').astype(int)

# purchases up to and including current row
events3['purchases_cum'] = events3.groupby('user_id')['is_purchase'].cumsum()

# TODO: leakage-safe: purchases strictly before current row
events3['purchases_before'] = ...

check('nonnegative', (events3['purchases_before'] >= 0).all())
print(events3[['user_id','event_ts','event','purchases_before']].head(10))


## Section 5 — Apply vs Vectorization

### Task 5.1: Remove slow apply

Create a `country_is_in` boolean column on `users`.
- First: do it with `.apply(...)` (slow style)
- Then: replace it with vectorized code

**Explain:** Why is `.apply` often slower in Pandas?


In [ ]:
u = users.copy()

# TODO: slow style (keep for comparison)
u['country_is_in_apply'] = u['country'].apply(lambda x: x == 'IN')

# TODO: vectorized style
u['country_is_in'] = ...

check('same_result', (u['country_is_in_apply'] == u['country_is_in']).all())
u


---
## Submission Checklist
- All TODOs completed
- All checks pass
- Explain prompts answered
